<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/crew_ai/CrewAI_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install required pacakges


In [1]:
!pip install --q crewai
!pip install --q -U duckduckgo-search
!pip install --q langchain_google_genai

### Required Packages

In [2]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import Agent, Task, Crew, Process

In [17]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
if GOOGLE_API_KEY :
  print("GOOGLE_API_KEY is set")

GOOGLE_API_KEY is set


### set up the LLM

In [31]:
# Set gemini pro as llm
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             verbose = True,
                             temperature = 0.6,
                             google_api_key=GOOGLE_API_KEY)

In [44]:
from crewai import Agent, LLM

gemini = LLM(
    api_key=os.getenv("GEMINI_API_KEY"),
    model="gemini/gemini-1.5-flash",
)

In [45]:
llm=gemini

### setup tools

In [33]:
# Make sure to Install duckduckgo-search for this example:
!pip install -U duckduckgo-search

In [34]:
!pip install --upgrade --quiet  langchain-community

In [35]:
# Make sure to Install duckduckgo-search for this example:
# !pip install -U duckduckgo-search

from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

In [54]:
from langchain_community.tools import DuckDuckGoSearchResults



### setup agents

In [56]:
from crewai.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
import os
import requests


class SearchTerm(BaseModel):
    """A search query"""
    message: str = Field(..., description="The message to be sent to the user.")

class SearchTool(BaseTool):


    name: str = "search something"
    description: str = (
        "This tool is used to do web search on duck duck go."
    )
    args_schema: Type[BaseModel] = SearchTerm

    def _run(self, message: str) -> str:
        tool = DuckDuckGoSearchResults()
        results = tool.invoke(message)
        print(results)
        return str(results.results)


In [57]:
# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=False,
  llm = llm,  #using google gemini pro API
  tools=[
        SearchTool()
      ]
  # You can pass an optional llm attribute specifying what mode you wanna use.
  # It can be a local model through Ollama / LM Studio or a remote
  # model like OpenAI, Mistral, Antrophic of others (https://python.langchain.com/docs/integrations/llms/)
  #
  # Examples:
  # llm=ollama_llm # was defined above in the file
  # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7)
)

In [58]:
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for
  your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=False,
  llm = llm,  #using google gemini pro API
  tools=[]
)


### Tasks to perform

In [59]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report""",
  agent=researcher,
  verbose=True,
  expected_output="A comprehensive analysis report of the latest advancements in AI in 2024, including key trends, breakthrough technologies, and potential industry impacts."
)

In [60]:
task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 4 paragraphs.""",
  agent=writer,
  expected_output="An engaging blog post of at least 4 paragraphs highlighting significant AI advancements, written in an accessible and cool tone without complex AI jargon."
)

### Create a Crew

In [61]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=False, # You can set it to 1 or 2 to different logging levels
)

In [62]:
crew

Crew(id=69c945ff-46bb-4050-aa89-02187906ff21, process=Process.sequential, number_of_agents=2, number_of_tasks=2)

### Kickoff the crew - let the magic happen

In [63]:
# Get your crew to work!
result = crew.kickoff()

# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report
snippet: In 2024, he demonstrated this when he was made joint winner of the Nobel Prize for chemistry, thanks to the AI model AlphaFold 2 being integral to the task of creating new proteins. By predicting ..., title: 6 Game-Changing AI Breakthroughs That Defined 2024 - Forbes, link: https://www.forbes.com/sites/bernardmarr/2024/12/16/6-game-changing-ai-breakthroughs-that-defined-2024/, snippet: The biggest AI trends of 2024 included accessibility, ethics, sustainability and regulation, with companies like Google, NVIDIA and Amazon in leading roles ... (VR/AR) technologies have been around for a few years now, but they reached new heights in 2024, driven by AI's growing ability to create immersive experiences. ... Industry in focus ..., titl

In [64]:
print(result)

## AI: It's Not Sci-Fi Anymore – The Coolest Advancements of 2024

Forget the robots taking over – AI in 2024 is way more exciting than that!  This year's breakthroughs aren't about killer machines; they're about making our lives easier, smarter, and more sustainable.  We're seeing a massive shift from clunky, general-purpose AI to specialized systems that are super efficient and surprisingly human-like.

One of the biggest game-changers is Generative AI.  Think of it as AI that can *create* things – from writing stories and poems to designing stunning images and even composing music.  These AI systems are getting better every day, needing less computing power and becoming more accessible to everyone. We're moving away from giant, all-purpose models towards smaller, specialized ones that are perfect for specific jobs, making them even more powerful and efficient.  Imagine an AI that's a master chef, creating the perfect recipe based on your fridge contents – that's the future we're bui

In [44]:
#test
from litellm import completion

resp = completion(
    model="gemini/gemini-2.5-flash",
    messages=[{"role": "user", "content": "What is the capital of France?"}],
    reasoning_effort="low",
)


In [20]:
resp

NameError: name 'resp' is not defined